# BiLSTM-CRF for PoS Tagging

Implementing CRF by [pytorch-crf](https://pytorch-crf.readthedocs.io/en/stable/). Install the package:  
```Python
$ pip install pytorch-crf
``` 

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True